In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
from scipy.spatial import distance
from itertools import chain, combinations
import matplotlib.pyplot as plt
import os, sys


In [2]:
class Data:
    def __init__(self):
        self.customerNum = 0 
        self.nodeNum     = 0 
        self.droneNum    = 2
        self.cities      = []
        self.cor_X       = [] 
        self.cor_Y       = [] 
        self.serviceTime = [] 
        self.disMatrix   = [[]]
        self.dt          = None
        self.i_pot = None
        self.cus_can_served_by_drone = None
        self.drone_distances = None
        self.truck_distances = None
        self.model = None
      
        

    def readData(self, path):
        self.dt = pd.read_csv(path, header = None).to_numpy()[:-1]
        self.customerNum = len(self.dt)
        self.i_pot = self.dt[0, 1:3]
        self.nodeNum = self.customerNum + 2 
        self.cor_X = [self.dt[i, 1:3][0] for i in range(len(self.dt))]
        self.cor_Y = [self.dt[i, 1:3][1] for i in range(len(self.dt))]
        self.cities = [self.dt[i, 0] for i in range(len(self.dt))]
        
        self.cus_can_served_by_drone = [i for i in range(len(self.dt)) if self.dt[i, 3] == 0]
        
        self.drone_distances = [round(distance.euclidean((self.dt[i, 1:3]), self.i_pot),2)
                                if self.dt[i, 3] == 0 else float('inf')
                                for i in range(len(self.dt))]
        self.truck_distances = [[round(distance.cityblock(self.dt[i, 1:3], self.dt[j, 1:3]),1)
                                 for i in range(len(self.dt))] for j in range(len(self.dt))]
    

        #Decision variables

        # x_ij if (i->j) in vehicle tour
        # # y_im = 1 if cus i assigned to drone m ()

        self.x = None  
        self.y = None 
    def addConstrs(self):


            #SET
            V = [i for i in range(self.customerNum)]
            C = [i for i in range (1, self.customerNum)]
            U = [k for k in range(self.droneNum)]
            G = [0] + C
            C_U = data.cus_can_served_by_drone


            C_truck = {(i,j): self.truck_distances[i][j] for i in V for j in V}
            C_drone = self.drone_distances

            self.x = [[[] for i in V] for j in V]  
            self.y = [[[] for i in V] for k in U] 
            self.u = [i for i in C] 
            print(self.u)
            
            #1
            #completion time
            alpha = self.model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, "traveltime")
            self.model.update()
            expr = LinExpr(0)
            expr.addTerms(1.0, alpha)

            self.model.setObjective(expr, GRB.MINIMIZE)
            expr.clear()

            #2
            expr = LinExpr(0)
            for i in V:
                for j in V:
                    if i != j:
                        self.x[i][j] = self.model.addVar(0, 1, vtype = GRB.BINARY,name = "x%d,%d" %(i,j))

                        self.model.update()
                        expr.addTerms(self.truck_distances[i][j], self.x[i][j])
                    else:
                        self.x[i][i] = self.model.addVar(0.0, 1.0, 0.0, GRB.BINARY, "x%d,%d" %(i,j))

    #         print(expr)

            self.model.addConstr(alpha >= expr, "truckTime")
            expr.clear()
            self.model.update()



            #3
            for k in U:
                expr = LinExpr(0)
                for i in C:

                    if i in C_U:
                        self.y[k][i] = self.model.addVar(0,1, vtype= GRB.BINARY, name =  "y%d,%d" %(k,i))
    #                     print(i)

                        expr.addTerms(self.drone_distances[i], self.y[k][i])
                    else:
                        self.y[k][i] = self.model.addVar(0, 0, vtype = GRB.BINARY, name = "y%d,%d" %(k,i))
                self.model.update()
    #             print(expr)
                self.model.addConstr(alpha >= expr, "dronetime")
                expr.clear()
            expr.clear()    


            #4
            for j in C:

                expr1 = LinExpr(0)
                expr2 = LinExpr(0)

                for i in V:

                    expr1.addTerms(1.0, self.x[i][j])
    #                 print(expr1)

                if j in C_U:
                    for k in U:
                        expr2.addTerms(1.0, self.y[k][j])

    #             print(expr2)
    #             print("------------------")

                self.model.addConstr(expr1 + expr2 == 1, "served customer once")
                expr1.clear()
                expr2.clear()
            expr1.clear()
            expr2.clear()


            #5
            for i in C:

                expr1 = LinExpr(0)
                expr2 = LinExpr(0)

                for j in V:

                    expr1.addTerms(1.0, self.x[i][j])
    #             print(expr1)
                if i in C_U:
                    for k in U:
                        expr2.addTerms(1.0, self.y[k][i])

    #             print(expr2)
    #             print("------------------")
                self.model.addConstr(expr1 + expr2 == 1, "served customer once")
                expr1.clear()
                expr2.clear()
            expr1.clear()
            expr2.clear()


            #6
            for i in C:
                expr1 = LinExpr(0)
                expr2 = LinExpr(0)

                for j in V:

                    expr1.addTerms(1.0, self.x[j][i])

                for h in V:

                    expr2.addTerms(1.0, self.x[i][h])
                self.model.addConstr(expr1 == expr2, "flow conservation")
                expr1.clear()
                expr2.clear()
            expr1.clear()
            expr2.clear()



            #get all subtours
            def powerset(iterable):
                "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
                s = list(iterable)
                return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

            S = list(powerset(range(1, len(data.cities))))
            # The first element of the list is the empty set and the last element is the full set, hence we remove them.
            S = S[1:(len(S))]
            S = [list(s) for s in S]

    #         print(len(S))
    #         import sys
    #         print(sys.getsizeof(S)/1024/1024," GB")
    #         print(type(S))



            # for s in S:

            #         s.insert(0,0)
            S.insert(0,[0])
            S = S[0:len(S) -1 ]
            # print(S)
            # S = [[0,1,2,3,4]]


            #7
            for s in S:
                expr1 = LinExpr(0)
                expr2 = LinExpr(0)

                for i in s:
                    for j in V:
                        if j not in s:
                            expr1.addTerms(1.0, self.x[i][j])

                    for k in U:
                        if i in C_U:
                            expr2.addTerms(1.0, self.y[k][i])
                self.model.update()
    #     print(expr1)
    #     print(expr2)
                self.model.addConstr(expr1 + expr2 >= 1)
                expr1.clear()
                expr2.clear()
                
            #8
            expr = LinExpr(0)
            for i in C:
                expr.addTerms(1.0, self.x[0][i])
            self.model.addConstr(expr == 1)
            expr.clear()        
            #9
            expr = LinExpr(0)
            for i in C:
                expr.addTerms(1.0, self.x[i][0])
            self.model.addConstr(expr == 1)
            expr.clear()

In [4]:
path = r'C:\\Users\\thebl\\Documents\\PDSTSP_MIP\PDSTSP_10_customer_problems'
path = "C:\\Users\\thebl\\Documents\\PDSTSP_MILP_new\\PDSTSP_10_customer_problems"
dirs = os.listdir(path)
problems_list = [file for file in dirs]

print(problems_list)

['20140813T111604.csv', '20140813T111606.csv', '20140813T111609.csv', '20140813T111611.csv', '20140813T111613.csv', '20140813T111615.csv', '20140813T111617.csv', '20140813T111619.csv', '20140813T111621.csv', '20140813T111623.csv', '20140813T111626.csv', '20140813T111628.csv', '20140813T111630.csv', '20140813T111632.csv', '20140813T111634.csv', '20140813T111636.csv', '20140813T111638.csv', '20140813T111640.csv', '20140813T111642.csv', '20140813T111644.csv', '20140813T111646.csv', '20140813T111649.csv', '20140813T111651.csv', '20140813T111653.csv', '20140813T111655.csv', '20140813T111657.csv', '20140813T111659.csv', '20140813T111701.csv', '20140813T111703.csv', '20140813T111705.csv', '20140813T111708.csv', '20140813T111710.csv', '20140813T111712.csv', '20140813T111714.csv', '20140813T111716.csv', '20140813T111718.csv', '20140813T111720.csv', '20140813T111722.csv', '20140813T111725.csv', '20140813T111727.csv', '20140813T111729.csv', '20140813T111731.csv', '20140813T111733.csv', '20140813T

In [5]:
data = Data()
data.model = Model("PDSTSP")
data.readData("C:\\Users\\thebl\\Documents\\PDSTSP_MILP_new\\PDSTSP_10_customer_problems\\" + problems_list[25])
data.addConstrs()

data.model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-12
[1, 2, 3, 4, 5, 6, 7, 8, 9]
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 543 rows, 119 columns and 15585 nonzeros
Model fingerprint: 0x2a7d6798
Variable types: 1 continuous, 118 integer (118 binary)
Coefficient statistics:
  Matrix range     [8e-01, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 152.5999999
Presolve removed 12 rows and 14 columns
Presolve time: 0.01s
Presolved: 531 rows, 105 columns, 15419 nonzeros
Found heuristic solution: objective 152.6000000
Variable types: 0 continuous, 105 integer (104 binary)

Root relaxation: objective 2.260022e+01, 48 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

In [6]:
data.model.printAttr('X')


    Variable            X 
-------------------------
  traveltime        36.59 
        x0,3            1 
        x1,0            1 
        x3,5            1 
        x5,8            1 
        x7,1            1 
        x8,7            1 
        y0,4            1 
        y0,6            1 
        y1,2            1 
        y1,9            1 


In [3]:
G = [i for i in range (0, data.customerNum)]
truck_tours = [(i,j) for i in G for j in G if data.x[i][j].x > 0.99]

U = [k for k in range(data.droneNum)]

C_U = data.cus_can_served_by_drone

plt.plot(data.i_pot[0], data.i_pot[1], c='r', marker='s')
# plt.scatter(xc[1:], yc[1:], c='b')
for i in range(1,data.customerNum):
    plt.scatter(data.cor_X, data.cor_Y, c='b')
    plt.annotate(str("  ") + str(int(data.cities[i])), (data.cor_X[i], data.cor_Y[i]))
    
for i, j in truck_tours:
    plt.plot([data.cor_X[i], data.cor_X[j]], [data.cor_Y[i], data.cor_Y[j]], c='g', zorder=0)
    
    
default_width = 1

for k in U:
    default_width+= 2
    for i in C_U:
        if data.y[k][i].x > 0.99:
            plt.plot([data.i_pot[0],data.cor_X[i]], [data.i_pot[1], data.cor_Y[i]], color='black', linestyle='dashed', linewidth = default_width)

NameError: name 'data' is not defined